In [2]:
from PLL import PhaseLockedLoop
import matplotlib.pyplot as plt
import numpy as np

class Fast_PhaseLockedLoop:
    def __init__( self, K_i, K_p, K_0, pll_f, fs ) -> None:     
        self.lastSinOut = 0
        
        self.fs = fs
        
        # LF State
        self.K_p = K_p
        self.K_i = K_i
        self.lf_integrator = 0
        
        # NCO State
        self.K_0 = K_0
        self.nco_f = pll_f
        self.nco_theta = 0
        self.nco_phaseEstimate = 0
        
    def proc( self, in_, ):
        # Phase Detector
        e_D = in_ * self.lastSinOut

        # Loop Filter
        self.lf_integrator += self.K_i * e_D / self.fs
        e_F = self.K_p*e_D + self.lf_integrator
        
        # NCO
        self.nco_theta += 2*np.pi*self.nco_f/self.fs + self.K_0 * e_F
        if self.nco_theta > 2*np.pi:
            self.nco_theta -= 2*np.pi
        
        cosOut = np.cos( self.nco_theta  )
        sinOut = -np.sin( self.nco_theta  )
        
        self.lastSinOut = sinOut
        return cosOut, sinOut, e_D, e_F

K = 1
duration_secs = 2 / K

for fs in [150 * K * 10, 150 * K * 100, 200 * K * 10]:
    in_f_Hz   = 10*K
    fs = 150 * K * 10

    t = np.linspace(0,duration_secs, int(duration_secs*fs))
    in_sig    = np.cos( 2*np.pi * in_f_Hz*1.25 * t + 0 )

    for K_p in [0.2667/10, 0.2667, 0.1337/10,]:
        for K_i in [0.0178*100, 0.0178, 0.0178*10, 0.1337]:
            for K_0 in [1, 2, 5]:
                for in_f_Hz in [ 9900 ]:
                    pll = PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )
                    fast_pll = Fast_PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )

                    for idx in range(len(in_sig)):
                        orig_cosOut, orig_sinOut, orig_e_D, orig_e_F = pll.proc( in_sig[idx] )

                        fast_cosOut, fast_sinOut, fast_e_D, fast_e_F = fast_pll.proc( in_sig[idx] )

                        assert( orig_cosOut == fast_cosOut )
                        assert( orig_sinOut == fast_sinOut )
                        assert( orig_e_D == fast_e_D )
                        assert( orig_e_F == fast_e_F )
                    
                    

print( "All passed!" )

All passed!


In [9]:
import time

K = 1
duration_secs = 10 / K

start = time.time()
for fs in [256000]:
    in_f_Hz   = 10*K

    t = np.linspace(0,duration_secs, int(duration_secs*fs))
    in_sig    = np.cos( 2*np.pi * in_f_Hz*1.25 * t + 0 )

    for K_p in [0.2667/10]:
        for K_i in [0.0178*100]:
            for K_0 in [1]:
                for in_f_Hz in [ 9900 ]:
                    pll = PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )

                    for idx in range(len(in_sig)):
                        orig_cosOut, orig_sinOut, orig_e_D, orig_e_F = pll.proc( in_sig[idx] )

end = time.time()
print( "original:", end-start )

start = time.time()
for fs in [256000]:
    in_f_Hz   = 10*K

    t = np.linspace(0,duration_secs, int(duration_secs*fs))
    in_sig    = np.cos( 2*np.pi * in_f_Hz*1.25 * t + 0 )

    for K_p in [0.2667/10]:
        for K_i in [0.0178*100]:
            for K_0 in [1]:
                for in_f_Hz in [ 9900 ]:
                    fast_pll = Fast_PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )

                    for idx in range(len(in_sig)):
                        fast_cosOut, fast_sinOut, fast_e_D, fast_e_F = fast_pll.proc( in_sig[idx] )
end = time.time()
print( "fast:", end-start )

original: 6.219636678695679
fast: 5.810209512710571
